In [9]:
# Basic libraries
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle

tf.reset_default_graph()
tf.set_random_seed(2016)
np.random.seed(2016)

# LSTM-autoencoder
from LSTMAutoencoder import *
print("\n\n\n\n")

rawdata = pickle.load(open("data/save.p", "rb"))

# Constants
batch_num = 77
hidden_num = 12
step_num = 8
elem_num = 1
iteration = 100000

#data = data[len(data)%step_num:]
data = tf.convert_to_tensor(rawdata, dtype=tf.float32)
# print(data.get_shape().as_list())
# placeholder list
p_input = data

# p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]
p_inputs = []
for t in tf.split(p_input, step_num, 1):
    # Ignoring squeezing since it removes the necessary shape (77, ?)
    p_inputs.append(t)

cell = tf.nn.rnn_cell.LSTMCell(hidden_num, use_peepholes=True)
ae = LSTMAutoencoder(hidden_num, p_inputs, cell=cell, decode_without_input=True)

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iteration):
        # Replace these random sequences
        """Random sequences.
          Every sequence has size batch_num * step_num * elem_num 
          Each step number increases 1 by 1.
          An initial number of each sequence is in the range from 0 to 19.
          (ex. [8. 9. 10. 11. 12. 13. 14. 15])
        """
        r = np.random.randint(20, size=batch_num).reshape([batch_num, 1, 1])
        r = np.tile(r, (1, step_num, elem_num))
        d = np.linspace(0, step_num, step_num, endpoint=False).reshape([1, step_num, elem_num])
        d = np.tile(d, (batch_num, 1, 1))

        random_sequences = rawdata
        
        #random_sequences = random_sequences[:, :, 0]
        
        (loss_val, _) = sess.run([ae.loss, ae.train], {p_input: random_sequences})
        if i % 1000 == 0:
            print('iter %d:' % (i), loss_val)
        if i % 10000 == 0:
            save_path = saver.save(sess, "/tmp/model.ckpt")
            print("Model saved in path: {}".format(save_path))
            
    random_sequences = rawdata
    #random_sequences = random_sequences[:, :, 0]
    
    (input_, output_) = sess.run([ae.input_, ae.output_], {p_input: random_sequences})
    print('train result :')
    print('input :', input_[0, :, :].flatten())
    print('output :', output_[0, :, :].flatten())







[[60.0, 365.0, 0.92, 54.91, 16.44, 54.91, 1.67, 1.0], [84.0, 2608.0, 0.74, 62.27, 3.22, 62.27, 1.19, 1.0], [220.0, 1095.0, 1.04, 228.26, 20.09, 76.09, 1.36, 3.0], [83.0, 732.0, 0.76, 62.94, 11.34, 0.0, 0.0, 0.0], [48.0, 888.0, 4.14, 198.7, 5.41, 198.7, 2.08, 1.0], [723.0, 31399.0, 0.61, 442.91, 2.3, 88.58, 0.69, 5.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [597.0, 2369.0, 1.19, 707.61, 25.2, 176.9, 0.67, 4.0], [3.0, 138.0, 3.92, 11.76, 2.17, 0.0, 0.0, 0.0], [18.0, 161.0, 3.69, 66.49, 11.18, 0.0, 0.0, 0.0], [1105.0, 5283.0, 0.95, 1048.87, 20.92, 348.65, 0.27, 3.0], [35.0, 1151.0, 4.53, 158.52, 3.04, 79.26, 5.71, 2.0], [2775.0, 362286.0, 0.29, 805.33, 0.77, 201.22, 0.14, 4.0], [294.0, 4999.0, 1.33, 391.59, 5.88, 391.59, 0.34, 1.0], [0.0, 160.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [111.0, 11513.0, 2.07, 229.27, 0.96, 0.0, 0.0, 0.0], [2100.0, 564666.0, 0.57, 1190.5, 0.37, 396.83, 0.14, 3.0], [11.0, 454.0, 1.1, 12.14, 2.42, 0.0, 0.0, 0.0], [134.0